In [6]:
import os
import subprocess
import torch
from torchvision import datasets, transforms
import torchvision.transforms
from PIL import Image
import json
import shutil
import split_dataset as split

from os import listdir
from os.path import isfile, join
import os
from PIL import Image   
from tqdm.notebook import tqdm


In [2]:
print(torch.version.cuda)
print(torch.__version__)

11.1
1.8.1


In [3]:
path_home = '/home/pathorad3090/Documents/Hadar/SyntheticEvaluation'
raw_data = path_home + "/data/mnist_images"

In [4]:
to_rgb = transforms.Lambda(lambda img: img.convert('RGB'))
# rgb_image = to_rgb(image)


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:01<00:00, 6927456.36it/s] 


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 187703.68it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:01<00:00, 1571950.69it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 1007484.73it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [7]:
# Create directories to save MNIST images
mnist_dir = raw_data
os.makedirs(mnist_dir, exist_ok=True)

# Download MNIST dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    # transforms.RGB(),
    to_rgb,
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)

# Save each image as a separate file
for idx, (img, label) in tqdm(enumerate(mnist_dataset), total=len(mnist_dataset)):
    img_path = os.path.join(mnist_dir, f'{label}_{idx}.png')
    img = transforms.ToPILImage()(img)
    img.save(img_path)



  0%|          | 0/60000 [00:00<?, ?it/s]

In [8]:


# raw_data = path_home + "/data/rgbs"


IMAGE_PATH = raw_data
print(IMAGE_PATH)
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
consistent_files = 0
inconsistent_size_files = 0
inconsistent_color_files = 0
total_files = len(files)

# Set the interval for status updates (e.g., every 100 files)
update_interval = int(total_files/5)

# Create a progress bar
progress_bar = tqdm(total=total_files, desc="Processing files", unit="file")

for i, file in enumerate(files, 1):
    file_path = os.path.join(IMAGE_PATH, file)
    img = Image.open(file_path)
    sz = img.size
    
    if base_size is None:
        base_size = sz
    
    if sz != base_size:
        inconsistent_size_files += 1
    elif img.mode != 'RGB':
        inconsistent_color_files += 1
    else:
        consistent_files += 1
    
    # Update the progress bar
    progress_bar.update(1)
    
    # Print status update every x files
    if i % update_interval == 0:
        print(f"Processed {i}/{total_files} files...")

# Close the progress bar
progress_bar.close()

# Print completion stats
print("\nProcessing complete. Summary:")
print(f"Total files processed: {total_files}")
print(f"Consistent files: {consistent_files}")
print(f"Files with inconsistent size: {inconsistent_size_files}")
print(f"Files with inconsistent color format: {inconsistent_color_files}")
print(f"Base size: {base_size}")



/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images


Processing files:   0%|          | 0/60000 [00:00<?, ?file/s]

Processed 12000/60000 files...
Processed 24000/60000 files...
Processed 36000/60000 files...
Processed 48000/60000 files...
Processed 60000/60000 files...

Processing complete. Summary:
Total files processed: 60000
Consistent files: 60000
Files with inconsistent size: 0
Files with inconsistent color format: 0
Base size: (32, 32)


In [ ]:
# print the color mode of 1 of the images


In [9]:
# divide into subdirectories

# def distribute_files(src_dir):
#     # Create class directories if they don't exist
#     for class_num in range(10):
#         class_dir = os.path.join(src_dir, f"{class_num}")
#         if not os.path.exists(class_dir):
#             os.makedirs(class_dir)

#     # Distribute files into their respective class directories
#     for filename in os.listdir(src_dir):
#         src_path = os.path.join(src_dir, filename)
#         if os.path.isfile(src_path):
#             try:
#                 file_class = filename.split("_")[0][0]
#                 if file_class.isdigit():
#                     dst_path = os.path.join(src_dir, f"{file_class}", filename)
#                     shutil.move(src_path, dst_path)
#             except (IndexError, ValueError):
#                 # Handle files that don't follow the expected naming convention
#                 print(f"Skipping file: {filename} (doesn't follow the expected naming convention)")

source_directory = raw_data
split.distribute_files_to_label_dirs(source_directory)

In [ ]:
# # Create initial dataset.json file from raw dataset:

# # {
# #     "labels":
# #         [
# #             ["folder1/1.jpg", 0], ["folder1/2.jpg", 0], ["folder1/3.jpg", 0], 
# #             ["folder2/4.jpg", 1], ["folder2/5.jpg", 1], ["folder2/6.jpg", 1], 
# #             ["folder3/7.jpg", 2], ["folder3/8.jpg", 2], ["folder3/9.jpg", 2], 
# #         ]
# # }

# def generate_labels_json(base_dir,output_dir, output_file_name):
#     print(f"Starting to generate labels JSON file...")
#     print(f"Base directory: {base_dir}")

#     labels_data = {"labels": []}
    
#     # Walk through the base directory and its subdirectories
#     for root, dirs, files in os.walk(base_dir):
#         for file in files:
#             # Only include image files (you can adjust this based on file extensions)
#             if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
#                 # Extract the class label from the first digit of the file name
#                 try:
#                     label = int(file[0])  # First digit of the file name
#                 except ValueError:
#                     # Skip files that do not start with a digit
#                     continue
                
#                 # Create relative file path from the base directory
#                 file_path = os.path.relpath(os.path.join(root, file), base_dir)
                
#                 # Add the file path and corresponding label to the list
#                 labels_data["labels"].append([file_path, label])


#     os.makedirs(output_dir, exist_ok=True)
#     output_file_path = os.path.join(output_dir, output_file_name)

#     # Write the labels to the output JSON file
#     with open(output_file_path, 'w') as f:
#         json.dump(labels_data, f, indent=4)

# base_directory = raw_data
# generate_labels_json(base_directory, base_directory, "dataset.json")


In [10]:
import os
import json

# Create initial dataset.json file from raw dataset:

base_directory = raw_data
# base_directory = '/Users/hadare/Documents/CodingProjects/SyntheticEvaluation/models/model_0K/images'
split.generate_labels_json(base_directory, path_home + '/data', "dataset_full.json")


Starting to generate labels JSON file...
Base directory: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images
Processed 10000 files...
Processed 20000 files...
Processed 30000 files...
Processed 40000 files...
Processed 50000 files...
Processed 60000 files...
Generated labels JSON file with 60000 entries.
